<a href="https://colab.research.google.com/github/hincz-lab/SCD-monitoring_ML-tool/blob/main/Image_Analysis_Colab_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone Repository, Change Working Directory, And Download Libraries/Packages.
Run the following cell one time. The SCD-monitoring_ML-tool Directory Will Populate In The User Interface To The Left (Click The Folder Icon And Go To "Content").

In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.
print("Cloning Repository...")
!git clone https://github.com/hincz-lab/SCD-monitoring_ML-tool &> /dev/null
print("Complete!")
print('==================================================================')
%cd SCD-monitoring_ML-tool
print("Is the New Working Directory.")
print('==================================================================')
print("Installing Necessary Libraries And Packages To Environment...")
!pip install -r requirement.txt &> /dev/null
print("Complete!")

## Import Tools Necessary To Run The Rest Of The Code.
Run the following cell one time. The necessary libraries and functions will be loaded from the downloaded set, as well as extracted from a google drive.

In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.
print("Importing...")
import tensorflow as tf
import keras as K
# load processing func
import source.toolkit_loc as tk
import source.dojo_tools as dj

from source.toolkit_loc import list_channels
from source.toolkit_loc import list_channels_df

# load main class object for monitoring blood cells
from source.SickleML_Monitor_Loc import CountAdheredBloodCells

# loading tools for extracting gdrive data
import source.load_data_tools_loc as loading_tools

import os
import pandas as pd
import time
from tqdm import tqdm
import sys
import gdown
from natsort import natsorted
print("Complete!")

## Download Pre-Trained Weights From Google Drive.
Run the following cell one time. Don't worry if it takes several minutes to load because there are several network weights to download. All of the pretrained weights will be saved in automatic folders called "./Phase2_Pretrained-models/" for phase 2 networks, or "./Phase1_Pretrained-models/" for phase 1 networks.

We expect seven networks in the phase 1 ensenble, and five networks in the phase 2 ensemble.

In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.

print("Downloading Phase 2 Weights...")
os.makedirs(f'./Phase2_Pretrained-models/Resnet50/', exist_ok = True)
os.chdir(f'./Phase2_Pretrained-models/Resnet50/') 

loading_tools.resnet50_gdrive_()
print("Complete!")
print('==================================================================')
os.chdir(f'../')
os.chdir(f'../')

print("Downloading Phase 1 Weights...")
os.makedirs(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/', exist_ok = True)
os.chdir(f'./Phase1_Pretrained-models/ce-jaccard_encoder-decoder-net/') 

loading_tools.ce_jaccard_gdrive_()
print("Complete!")
os.chdir(f'../')
os.chdir(f'../')

## Create Directory For Holding Channels To Be Analyzed.
Here you will create a directory which will hold all of the images that will be analyzed by the network.

It is important to note here that you will need to manually upload images to the correct directory, which is located at:
content -> SCD-monitoring_ML-tool -> data -> Images_To_Be_Analyzed.

To upload images, right-click the Images_To_Be_Analyzed folder and select "Upload". Then, navigate to and select whole channel images which you would like to be analyzed.

Make sure to wait until all images are properly uploaded before executing further code (You can see the status as an orange ring in the bottom-left corner of the screen).

IMPORTANT: It is recommended to do your analysis in batches, instead of uploading all whole channel images at once. After completing one batch, simply re-run the following cell, which will clear out the Images_To_Be_Analyzed directory, and then you can upload more channels for analysis.


In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.
' Download channel images from gdrive ... '
print("Creating Image Analysis Directory...")
import shutil
channel_path= './data/Images_To_Be_Analyzed/'
if os.path.isdir(channel_path):
  shutil.rmtree(channel_path)
os.makedirs(channel_path, exist_ok=True)
print("Complete!")

## Import Other Useful Functions.
Run the following cell once. The code will import other useful functions for the analysis of channels.

In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.
'  Here, we have a function that automatically loads the whole ensemble model (5 neural networks) '

print("Loading Useful Functions...")
def load_ensembles(Phase1_path_model, Phase2_path_model):

    Phase1_path = './Phase1_Pretrained-models/' + Phase1_path_model + '/'# folder for Phase I 
    Phase2_path = './Phase2_Pretrained-models/' + Phase2_path_model + '/'# folder for Phase II

    Phase1_ensemble = tk.load_zoo(Phase1_path) # loading the Phase I ensemble (expect: 7)
    Phase2_ensemble = tk.load_zoo(Phase2_path) # loading the Phase I ensemble (expect: 5)
    return Phase1_ensemble, Phase2_ensemble

' function for creating dataframes while computing cell counts during inference '

def create_final_df(counts, times):
    counts_df = pd.DataFrame(counts)
    counts_df.columns = ['filename', 'sRBC', 'WBC', 'Other']
    times_df = pd.DataFrame(times)
    times_df.columns = ['time_secs']
    final_df = pd.concat([counts_df, times_df], axis = 1)
    return final_df
print("Complete!")


## Make Predictions.
In the following cell, all whole channel images located in the Images_To_Be_Analyzed directory will be sent through the neural-network. Cells will be extracted and guesses will be made as to the identity of each. 

The output, which is seen by executing the second of the following two cells, will be in the form of a table, where counts of sRBC, WBC, and Other will be given for each whole channel image.

In [ ]:
#@title <-- Click To Run Cell. Double Click Here To View Code.
%%time 



counts, times = [], [] 
count_container, time_container = [], []

sRBC_Thresh = [0.4]
WBC_Thresh = [0.4]
Other_Thresh = [0.9]

Phase1_names, Phase2_names = 'ce-jaccard_encoder-decoder-net', 'Resnet50'

print("Loading Ensembles...")
Phase1_ensemble, Phase2_ensemble = load_ensembles(Phase1_names, Phase2_names)
print("Complete!")
counts, times = [], [] 
print('==================================================================')

for index, filenames in enumerate(os.listdir(channel_path)):
    for rep in ((".png", ""), (".jpg", "")):
        clean_filename = filenames.replace(*rep)
    if clean_filename == ".ipynb_checkpoints":
      continue
    print('Analysis:', index, '| Channel:', clean_filename)
    print('==================================================================')
    channel = CountAdheredBloodCells(channel_path, filenames) # calling the class object
    # calling the function to output cell counts
    start = time.time()
    sRBC, WBC, Others = channel.call_pipeline(Phase1_ensemble, Phase2_ensemble, sRBC_Thresh, WBC_Thresh, Other_Thresh)
    end = time.time()
    run_time = end-start
            
    times.append([run_time])
    counts.append([filenames, sRBC, WBC, Others])
            
    final_df = create_final_df(counts,times)
   # final_df.to_csv(f'./AI-vs-Human_counts/{Phase1_name}_{Phase2_name}.csv', index = False)

In [ ]:
#@title <-- Click here to run cell. Double click here to view code.
print("Here are the predicted counts ...")
final_df